In [24]:
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from metpy.units import units
from netCDF4 import num2date
import numpy as np
import scipy.ndimage as ndimage
from siphon.ncss import NCSS
from pyproj import Proj, transform
import utm
from warnings import filterwarnings
from scipy.interpolate import griddata

In [2]:
# Helper function for finding proper time variable
def find_time_var(var, time_basename='time'):
    for coord_name in var.coordinates.split():
        if coord_name.startswith(time_basename):
            return coord_name
    raise ValueError('No time variable found for ' + var.name)

In [3]:
base_url = 'https://www.ncei.noaa.gov/thredds/ncss/model-rap130anl-old/'
dt = datetime(2016, 1, 1, 12)
ncss = NCSS('{}{dt:%Y%m}/{dt:%Y%m%d}/rap_130_{dt:%Y%m%d}'
            '_{dt:%H}00_000.grb2'.format(base_url, dt=dt))

# Create lat/lon box for location you want to get data for
query = ncss.query().time(dt)
query.lonlat_box(north=36.5, south=25.8, east=-93.5, west=-106.65)
query.accept('netcdf')

# Request data for model "surface" data
query.variables('Temperature_height_above_ground',
                'u-component_of_wind_height_above_ground',
                'v-component_of_wind_height_above_ground')
data = ncss.get_data(query)

In [4]:
data.variables.keys()

dict_keys(['v-component_of_wind_height_above_ground', 'time', 'height_above_ground4', 'y', 'x', 'LambertConformal_Projection', 'u-component_of_wind_height_above_ground', 'Temperature_height_above_ground', 'height_above_ground1'])

The variables come from different levels and may have different data array shapes.

80m Temp: Temperature_height_above_ground (time, level, lat, lon)
80m Wind: u/v-component_of_wind_height_above_ground (time, level, lat, lon)

Height above ground Temp from RAP has 2 levelS (2m, 80m)
Height above ground Wind from RAP has 2 levels (10m, 80m)

In [120]:
filterwarnings("ignore", category=DeprecationWarning) 

# Pull out variables you want to use
temp = units.K * data.variables['Temperature_height_above_ground'][:].squeeze()
lev_80m_temp = np.where(data.variables['height_above_ground1'][:] == 80)[0][0]
temp_80m = temp[lev_80m_temp]
u_wind = units('m/s') * data.variables['u-component_of_wind_height_above_ground'][:].squeeze()
v_wind = units('m/s') * data.variables['v-component_of_wind_height_above_ground'][:].squeeze()
x = data.variables['x'][:].squeeze()
y = data.variables['y'][:].squeeze()
time_var = data.variables[find_time_var(data.variables['Temperature_height_above_ground'])]

# Convert number of hours since the reference time into an actual date
time = num2date(time_var[:].squeeze(), time_var.units)

lev_80m_wind = np.where(data.variables['height_above_ground4'][:] == 80)[0][0]
u_wind_80m = u_wind[lev_80m_wind]
v_wind_80m = v_wind[lev_80m_wind]

# Convert masked arrays to regular numpy arrays
x = x.compressed()
y = y.compressed()

# Combine 1D x and y coordinates into a 2D grid of locations
# x_2d, y_2d = np.meshgrid(x, y)
x_2d, y_2d = np.meshgrid(x, y, sparse=True)
# x_2d, y_2d = np.mgrid[x, y]

# Get subset of valid points for which we have data
x_valid_temp = x_2d[~temp_80m.mask].ravel()
x_valid_u = x_2d[~u_wind_80m.mask].ravel()
x_valid_v = x_2d[~v_wind_80m.mask].ravel()

y_valid_temp = y_2d[~temp_80m.mask].squeeze()
y_valid_u = y_2d[~u_wind_80m.mask].squeeze()
y_valid_v = y_2d[~v_wind_80m.mask].squeeze()

# Get 2D grid array from coordinate vectors
# x_valid_temp, y_valid_temp = np.meshgrid(x_valid_temp,y_valid_temp)
# x_valid_u, y_valid_u = np.meshgrid(x_valid_u,y_valid_u)
# x_valid_v, y_valid_v = np.meshgrid(x_valid_v,y_valid_v)

temp_80m_valid = temp_80m[~temp_80m.mask].squeeze()
u_wind_valid = u_wind_80m[~u_wind_80m.mask].squeeze()
v_wind_valid = v_wind_80m[~v_wind_80m.mask].squeeze()

# Transpose data to get to (x,y) order
temp_80m_valid = np.transpose(temp_80m_valid)
u_wind_valid = np.transpose(u_wind_valid)
v_wind_valid = np.transpose(v_wind_valid)

In [118]:
x_valid_temp.shape

(98,)

In [121]:
# Convert masked weather arrays to numpy arrays by interpolating to fill missing data points
temp_80m_interp = griddata((x_valid_temp,y_valid_temp), temp_80m_valid, (x_2d,y_2d), method='nearest')
u_80m_interp = griddata((x_valid_u,y_valid_u), u_wind_valid, (x_2d,y_2d), method='nearst')
v_80m_interp = griddata((x_valid_u,y_valid_u), v_wind_valid, (x_2d,y_2d), method='nearest')

ValueError: shape mismatch: objects cannot be broadcast to a single shape

In [ ]:
# Convert x and y from lambert conformal conic projection to lat-lon
inProj = Proj(init='epsg:2154')
outProj = Proj(init='epsg:4326')
lat,lon = transform(inProj,outProj,x,y)

# Convert lat-lon to utm projection
x,y = utm.from_latlon()

# Convert masked arrays to regular numpy arrays
x = x.compressed()
y = y.compressed()


Begin map creation
------------------



In [ ]:
# Set Projection of Data
datacrs = ccrs.PlateCarree()

# Set Projection of Plot
plotcrs = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)

# Create new figure
fig = plt.figure(figsize=(11, 8.5))

# Add the map and set the extent
ax = plt.subplot(111, projection=plotcrs)
plt.title('GFS Analysis MSLP, 2m Temperature (F), Wind Barbs (kt)'
          ' {0:%d %B %Y %H:%MZ}'.format(time), fontsize=16)
ax.set_extent([235., 290., 20., 55.])

# Add state boundaries to plot
states_provinces = cfeature.NaturalEarthFeature(category='cultural',
                                                name='admin_1_states_provinces_lakes',
                                                scale='50m', facecolor='none')
ax.add_feature(states_provinces, edgecolor='black', linewidth=1)

# Add country borders to plot
country_borders = cfeature.NaturalEarthFeature(category='cultural',
                                               name='admin_0_countries',
                                               scale='50m', facecolor='none')
ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Plot MSLP Contours
clev_mslp = np.arange(0, 1200, 4)
cs = ax.contour(lon_2d, lat_2d, smooth_mslp, clev_mslp, colors='black', linewidths=1.5,
                linestyles='solid', transform=datacrs)
plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)

# Plot 2m Temperature Contours
clevtemp = np.arange(-60, 101, 10)
cs2 = ax.contour(lon_2d, lat_2d, temp.to(units('degF')), clevtemp,
                 colors='tab:red', linewidths=1.25, linestyles='dotted',
                 transform=datacrs)
plt.clabel(cs2, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)

# Plot 10m Wind Barbs
ax.barbs(lon_2d, lat_2d, u_wind_10m.magnitude, v_wind_10m.magnitude,
         length=6, regrid_shape=20, pivot='middle', transform=datacrs)

plt.show()